# **Exercícios Resolvidos de SQL**
- Testado em PostgreSQL 10.19
- Veja mais detalhes em: [banco-de-dados](https://github.com/brunocampos01/banco-de-dados)
- É possivel executar todos os comandos dentro do colab notebook usando somente o browser

## **Preparação de Ambiente**

In [ ]:
# install postgresql
!apt install postgresql postgresql-contrib &>log

# start server
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

 * Starting PostgreSQL 10 database server
   ...done.
CREATE ROLE


In [ ]:
# setup client with magic
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

'Connected: @postgres'

In [ ]:
%%sql
SELECT version();

 * postgresql+psycopg2://@/postgres


,version
0,PostgreSQL 10.22 (Ubuntu 10.22-0ubuntu0.18.04....


<br/>
<br/>

---

# **DDL: comandos para implementar o modelo lógico**

## **1. Crie um BD com nome `clinica`**

In [ ]:
%%sql
DROP TABLE IF EXISTS ambulatorios CASCADE;
DROP TABLE IF EXISTS medicos CASCADE;
DROP TABLE IF EXISTS pacientes CASCADE;
DROP TABLE IF EXISTS funcionarios CASCADE;
DROP TABLE IF EXISTS consultas CASCADE;

DROP DATABASE IF EXISTS clinica;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%%sql
CREATE DATABASE clinica;

 * postgresql+psycopg2://@/postgres


""


<br/>
<br/>

## **2. Crie as seguintes tabelas neste BD**
- Considere que os atributos em negrito são chaves primárias (PK)
e os em itálico são chaves estrangeiras (FK)
- O campo `cpf` não será utilizado em cálculos
- medicos não são funcionários da clinica

- **ambulatorios** 
  - **nroa (int)**
  - andar (numeric(3)) (não nulo)
  - capacidade (smallint)

In [ ]:
%%sql
CREATE TABLE ambulatorios(
  nroa 				INT 				PRIMARY KEY,
	andar 			NUMERIC(3)	NOT NULL,
	capacidade  SMALLINT
);

SELECT
	column_name, data_type
FROM information_schema.columns
WHERE 
  table_name = 'ambulatorios';

 * postgresql+psycopg2://@/postgres


,column_name,data_type
0,nroa,integer
1,andar,numeric
2,capacidade,smallint


- **medicos**
  - **cod_m (int)**
  - nome (varchar(40)) (não nulo)
  - idade (smallint) (não nulo)
  - especialidade (char(20))
  - CPF (varchar(11)) (único)
  - cidade (varchar(30))
  - _nroa (int)_

In [ ]:
%%sql
CREATE TABLE medicos(
	cod_m         INTEGER     PRIMARY KEY,
	nome          VARCHAR(40) NOT NULL,
	idade         SMALLINT    NOT NULL,
	especialidade CHAR(20),
	cpf           VARCHAR(11) UNIQUE,
	cidade        VARCHAR(30),
	nroa          INT,
	FOREIGN KEY (nroa) REFERENCES ambulatorios (nroa) 
  -- Se usar foreign key tem que ter a coluna nas 2 tabelas!
  -- A forma recomendada de se criar PK, FK, NOT NULL, CHECK, UNIQUE eh com contraint (aula 06)
	-- Example: CONSTRAINT fk_ambulatorio_nroa FOREIGN KEY (nroa) REFERENCES ambulatorios
);

SELECT 
	column_name, data_type
FROM information_schema.columns
WHERE 
  table_name = 'medicos';

 * postgresql+psycopg2://@/postgres


,column_name,data_type
0,cod_m,integer
1,nome,character varying
2,idade,smallint
3,especialidade,character
4,cpf,character varying
5,cidade,character varying
6,nroa,integer


In [ ]:
%%sql
select * from medicos 


 * postgresql+psycopg2://@/postgres


""


- **pacientes**
  - **cod_p (int)**
  - nome (varchar(40)) (não nulo)
  - idade (smallint) (não nulo)
  - cidade (char(30))
  - CPF (numeric(11)) (único)
  - doenca (varchar(40)) (não nulo)

In [ ]:
%%sql
CREATE TABLE pacientes(
	cod_p     INT         PRIMARY KEY,
	nome      VARCHAR(40) NOT NULL,
	idade     SMALLINT    NOT NULL,
	cidade    CHAR(30),
	cpf       VARCHAR(11) UNIQUE,
	doenca    VARCHAR(40) NOT NULL
);

SELECT 
	column_name, data_type
FROM information_schema.columns
WHERE 
  table_name = 'pacientes';

 * postgresql+psycopg2://@/postgres


,column_name,data_type
0,cod_p,integer
1,nome,character varying
2,idade,smallint
3,cidade,character
4,cpf,character varying
5,doenca,character varying


- **funcionarios**
  - **cod_f (int)**
  - nome (varchar(40)) (não nulo)
  - idade (smallint)
  - CPF (varchar(11)) (único)
  - cidade (varchar(30))
  - salario (numeric(10))
  - cargo (varchar(20))

In [ ]:
%%sql
CREATE TABLE funcionarios(
	cod_f     INT PRIMARY KEY,
	nome      VARCHAR(40) NOT NULL,
	idade     SMALLINT,
	cpf       VARCHAR(11) UNIQUE,
	cidade    VARCHAR(30),
	salario   NUMERIC(10),
	cargo     VARCHAR(20)
);

SELECT 
	column_name, data_type
FROM information_schema.columns
WHERE 
  table_name = 'funcionarios';

 * postgresql+psycopg2://@/postgres


,column_name,data_type
0,cod_f,integer
1,nome,character varying
2,idade,smallint
3,cpf,character varying
4,cidade,character varying
5,salario,numeric
6,cargo,character varying


- **consultas**
  - *cod_m (int)*
  - *cod_p (int)*
  - data (date) (não nulo)
  - hora (time) (não nulo)

In [ ]:
%%sql
CREATE TABLE consultas(
  cod_c     INT PRIMARY KEY,
	cod_m     INT,
	cod_p     INT,
	data      DATE NOT NULL,
	hora      TIME NOT NULL,
  FOREIGN KEY (cod_m) REFERENCES medicos (cod_m),
	FOREIGN KEY (cod_p) REFERENCES pacientes (cod_p)
);

SELECT 
	column_name, data_type
FROM information_schema.columns
WHERE 
  table_name = 'consultas';

 * postgresql+psycopg2://@/postgres


,column_name,data_type
0,cod_c,integer
1,cod_m,integer
2,cod_p,integer
3,data,date
4,hora,time without time zone


**NOTE:** a PK da tabela consultas também poderia ter sido modelada com chave composta

<br/>
<br/>

## **3. Crie os seguintes índices**
- Medicos: nroa
- Pacientes: doenca

In [ ]:
%%sql
CREATE UNIQUE INDEX nroa ON medicos(nroa);

SELECT * 
FROM pg_indexes 
WHERE tablename = 'medicos';

 * postgresql+psycopg2://@/postgres


,schemaname,tablename,indexname,tablespace,indexdef
0,public,medicos,medicos_pkey,None,CREATE UNIQUE INDEX medicos_pkey ON public.med...
1,public,medicos,medicos_cpf_key,None,CREATE UNIQUE INDEX medicos_cpf_key ON public....
2,public,medicos,nroa,None,CREATE UNIQUE INDEX nroa ON public.medicos USI...


In [ ]:
%%sql
CREATE UNIQUE INDEX doenca ON pacientes(doenca);

SELECT * 
FROM pg_indexes 
WHERE tablename = 'pacientes';

 * postgresql+psycopg2://@/postgres


,schemaname,tablename,indexname,tablespace,indexdef
0,public,pacientes,pacientes_pkey,None,CREATE UNIQUE INDEX pacientes_pkey ON public.p...
1,public,pacientes,pacientes_cpf_key,None,CREATE UNIQUE INDEX pacientes_cpf_key ON publi...
2,public,pacientes,doenca,None,CREATE UNIQUE INDEX doenca ON public.pacientes...


<br/>
<br/>


## **4. Remova o índice doenca em Pacientes**


In [ ]:
%%sql
DROP INDEX doenca;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%%sql
SELECT * 
FROM pg_indexes 
WHERE tablename = 'pacientes';

 * postgresql+psycopg2://@/postgres


,schemaname,tablename,indexname,tablespace,indexdef
0,public,pacientes,pacientes_pkey,None,CREATE UNIQUE INDEX pacientes_pkey ON public.p...
1,public,pacientes,pacientes_cpf_key,None,CREATE UNIQUE INDEX pacientes_cpf_key ON publi...


<br/>
<br/>

## **5. Remova a coluna `cargo` da tabela de `funcionarios`**

In [ ]:
%%sql
ALTER TABLE funcionarios
	DROP COLUMN cargo;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%%sql
SELECT 
	column_name, data_type
FROM information_schema.columns
WHERE 
  table_name = 'funcionarios';

 * postgresql+psycopg2://@/postgres


,column_name,data_type
0,cod_f,integer
1,nome,character varying
2,idade,smallint
3,cpf,character varying
4,cidade,character varying
5,salario,numeric


<br/>
<br/>

## **6. Delete todas as tabelas e reescreva seguindo a forma recomendada, com `CONSTRAINTS`**

In [ ]:
%%sql
DROP TABLE IF EXISTS ambulatorios CASCADE;
DROP TABLE IF EXISTS medicos CASCADE;
DROP TABLE IF EXISTS pacientes CASCADE;
DROP TABLE IF EXISTS funcionarios CASCADE;
DROP TABLE IF EXISTS consultas CASCADE;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%%sql

CREATE TABLE ambulatorios(
	nroa        INT,
	andar       NUMERIC(3)    NOT NULL,
	capacidade  SMALLINT,
  CONSTRAINT  PK_nroa	      PRIMARY KEY   (nroa)
);

CREATE TABLE medicos(
	cod_m         INTEGER,
	nome          VARCHAR(40)   NOT NULL,
	idade         SMALLINT      NOT NULL,
	especialidade CHAR(20),
	cpf           VARCHAR(11)   UNIQUE,
	cidade        VARCHAR(30),
	nroa          INTEGER,
  CONSTRAINT    PK_cod_m      PRIMARY KEY   (cod_m),
 	CONSTRAINT    FK_med_amb    
    FOREIGN KEY   (nroa) 
    REFERENCES ambulatorios (nroa)
);

CREATE TABLE pacientes(
	cod_p       INT,
	nome        VARCHAR(40)   NOT NULL,
	idade       SMALLINT      NOT NULL,
	cidade      CHAR(30),
	cpf         VARCHAR(11)   UNIQUE,
	doenca      VARCHAR(40)   NOT NULL,
  CONSTRAINT  PK_cod_p      PRIMARY KEY   (cod_p)
);

CREATE TABLE funcionarios(
	cod_f       INT,
	nome        VARCHAR(40)   NOT NULL,
	idade       SMALLINT,
	cpf         VARCHAR(11)   UNIQUE,
	cidade      VARCHAR(30),
	salario     NUMERIC(10),
	cargo       VARCHAR(20),
  CONSTRAINT  PK_cod_f      PRIMARY KEY   (cod_f)
);

CREATE TABLE consultas(
  cod_c           INT,
	cod_m           INT,
	cod_p           INT,
	data DATE       NOT NULL,
	hora TIME       NOT NULL,
  CONSTRAINT    PK_cod_c      PRIMARY KEY   (cod_c),
	CONSTRAINT      FK_consultas_medicos    
    FOREIGN KEY (cod_m) 
    REFERENCES medicos (cod_m),
	CONSTRAINT      FK_consultas_pacientes  
    FOREIGN KEY (cod_p) 
    REFERENCES pacientes (cod_p)
);

 * postgresql+psycopg2://@/postgres


""


**NOTE:**
Para evitar referências à tabelas ainda não criadas pode ser creiada todas as tabelas sem chaves estrangeiras (FK) e depois inserir a `CONSTRAINT` de FK com `ALTER TABLE`:

```sql
ALTER TABLE nomeTabela 
  ADD CONSTRAINT nome    
  FOREIGN KEY (nomeColunaChaveEstrangeira)    
  REFERENCES tabela (nomeColunaChavePrimaria);
```

<br/>
<br/>

---

# **DML: comandos para manipular os dados**
- insert
- alter
- update
- delete from
- select

## **7. Insira os seguites dados no DB**

Pode ser feito de 2 formas:
- Sem informar os atributos 
- Informando os atributos

In [ ]:
%%sql

--ambulatorios
INSERT INTO ambulatorios(nroa, andar, capacidade)
VALUES (1, 1, 30);

INSERT INTO ambulatorios(nroa, andar, capacidade)
VALUES (2, 1, 50);

INSERT INTO ambulatorios(nroa, andar, capacidade)
VALUES (3, 2, 40);

INSERT INTO ambulatorios(nroa, andar, capacidade)
VALUES (4, 2, 25);

INSERT INTO ambulatorios(nroa, andar, capacidade)
VALUES (5, 2, 55);

SELECT * FROM ambulatorios;

 * postgresql+psycopg2://@/postgres


,nroa,andar,capacidade
0,1,1,30
1,2,1,50
2,3,2,40
3,4,2,25
4,5,2,55


In [ ]:
%%sql

--medicos
INSERT INTO medicos(cod_m, nome, idade, especialidade, cpf,	cidade, nroa)
VALUES (1, 'joao', 40, 'ortopedia', '10000100000', 'florianopolis', 1);

INSERT INTO medicos(cod_m, nome, idade, especialidade, cpf,	cidade, nroa)
VALUES (2, 'maria', 42, 'traumatologia', '10000100001', 'blumenau', 2);

INSERT INTO medicos(cod_m, nome, idade, especialidade, cpf,	cidade, nroa)
VALUES (3, 'pedro', 51, 'pediatria', '10000100002', 'sao jose', 3);

INSERT INTO medicos(cod_m, nome, idade, especialidade, cpf,	cidade, nroa)
VALUES (4, 'carlos', 28, 'ortopedia', '10000100003', 'joinville', 4);

INSERT INTO medicos(cod_m, nome, idade, especialidade, cpf,	cidade, nroa)
VALUES (5, 'marcia', 33, 'neurologia', '10000100004', 'biguacu', 5);

SELECT * FROM medicos;

 * postgresql+psycopg2://@/postgres


,cod_m,nome,idade,especialidade,cpf,cidade,nroa
0,1,joao,40,ortopedia,10000100000,florianopolis,1
1,2,maria,42,traumatologia,10000100001,blumenau,2
2,3,pedro,51,pediatria,10000100002,sao jose,3
3,4,carlos,28,ortopedia,10000100003,joinville,4
4,5,marcia,33,neurologia,10000100004,biguacu,5


In [ ]:
%%sql

-- pacientes
INSERT INTO pacientes(cod_p, nome, idade,	cidade,	cpf, doenca)
VALUES (1, 'ana', 20, 'florianopolis', '12345678910', 'gripe');

INSERT INTO pacientes(cod_p, nome, idade,	cidade,	cpf, doenca)
VALUES (2, 'paulo', 24, 'palhoca', '12345678911', 'fratura');

INSERT INTO pacientes(cod_p, nome, idade,	cidade,	cpf, doenca)
VALUES (3, 'lucia', 30, 'biguacu', '12345678912', 'tendinite');

INSERT INTO pacientes(cod_p, nome, idade,	cidade,	cpf, doenca)
VALUES (4, 'carlos', 28, 'joinville', '12345678913', 'sarampo');

SELECT * FROM pacientes;

 * postgresql+psycopg2://@/postgres


,cod_p,nome,idade,cidade,cpf,doenca
0,1,ana,20,florianopolis,12345678910,gripe
1,2,paulo,24,palhoca,12345678911,fratura
2,3,lucia,30,biguacu,12345678912,tendinite
3,4,carlos,28,joinville,12345678913,sarampo


In [ ]:
%%sql

--funcionarios
INSERT INTO funcionarios (cod_f, nome, idade, cidade, salario, cpf)
VALUES (1, 'rita', 32, 'sao jose', 1200, '12345678920');

INSERT INTO funcionarios (cod_f, nome, idade, cidade, salario, cpf)
VALUES (2, 'maria', 55, 'palhoca', 1220, '12345678921');

INSERT INTO funcionarios (cod_f, nome, idade, cidade, salario, cpf)
VALUES (3, 'caio', 45, 'florianopolis', 1100, '12345678922');

INSERT INTO funcionarios (cod_f, nome, idade, cidade, salario, cpf)
VALUES (4, 'carlos', 44, 'florianopolis', 1200, '12345678923');

INSERT INTO funcionarios (cod_f, nome, idade, cidade, salario, cpf)
VALUES (5, 'paula', 33, 'florianopolis', 2500, '12345678924');

SELECT * FROM funcionarios;

 * postgresql+psycopg2://@/postgres


,cod_f,nome,idade,cpf,cidade,salario,cargo
0,1,rita,32,12345678920,sao jose,1200,None
1,2,maria,55,12345678921,palhoca,1220,None
2,3,caio,45,12345678922,florianopolis,1100,None
3,4,carlos,44,12345678923,florianopolis,1200,None
4,5,paula,33,12345678924,florianopolis,2500,None


In [ ]:
%%sql

--consultas
-- NOTE: DATE format: yyyy-mm-dd
INSERT INTO consultas
VALUES (1, 1, 1, '2016-10-12', '1400');

INSERT INTO consultas
VALUES (2, 1, 4, '2016-10-13', '1000');

INSERT INTO consultas
VALUES (3, 2, 1, '2016-10-13', '0900');

INSERT INTO consultas
VALUES (4, 2, 2, '2016-10-14', '1400');

INSERT INTO consultas
VALUES (5, 2, 4, '2016-10-14', '1700');

INSERT INTO consultas
VALUES (6, 3, 1, '2016-10-19', '1800');

INSERT INTO consultas
VALUES (7, 3, 3, '2016-10-12', '1000');

INSERT INTO consultas
VALUES (8, 3, 4, '2016-10-19', '1300');

INSERT INTO consultas
VALUES (9, 4, 4, '2016-10-22', '1930');

SELECT * FROM consultas;

 * postgresql+psycopg2://@/postgres


,cod_c,cod_m,cod_p,data,hora
0,1,1,1,2016-10-12,14:00:00
1,2,1,4,2016-10-13,10:00:00
2,3,2,1,2016-10-13,09:00:00
3,4,2,2,2016-10-14,14:00:00
4,5,2,4,2016-10-14,17:00:00
5,6,3,1,2016-10-19,18:00:00
6,7,3,3,2016-10-12,10:00:00
7,8,3,4,2016-10-19,13:00:00
8,9,4,4,2016-10-22,19:30:00


<br/>
<br/>

## **8. Execute as seguites alterações nos dados**

O paciente Paulo mudou-se para Ilhota

In [ ]:
%%sql
SELECT * FROM pacientes;

 * postgresql+psycopg2://@/postgres


,cod_p,nome,idade,cidade,cpf,doenca
0,1,ana,20,florianopolis,12345678910,gripe
1,2,paulo,24,palhoca,12345678911,fratura
2,3,lucia,30,biguacu,12345678912,tendinite
3,4,carlos,28,joinville,12345678913,sarampo


In [ ]:
%%sql
UPDATE pacientes SET cidade = 'ilhota' 
WHERE nome = 'paulo';

SELECT * FROM pacientes;

 * postgresql+psycopg2://@/postgres


,cod_p,nome,idade,cidade,cpf,doenca
0,1,ana,20,florianopolis,12345678910,gripe
1,3,lucia,30,biguacu,12345678912,tendinite
2,4,carlos,28,joinville,12345678913,sarampo
3,2,paulo,24,ilhota,12345678911,fratura


A consulta do médico 1 com o paciente 4 passou para às 12:00 horas do dia 4 de Novembro de 2016


In [ ]:
%%sql
SELECT * 
FROM consultas 
WHERE cod_m = 1;

 * postgresql+psycopg2://@/postgres


,cod_c,cod_m,cod_p,data,hora
0,1,1,1,2016-10-12,14:00:00
1,2,1,4,2016-10-13,10:00:00


In [ ]:
%%sql
UPDATE consultas SET hora='1200', data='2016-11-04' 
WHERE cod_m = 1 AND cod_p = 4;

SELECT * 
FROM consultas 
WHERE cod_m=1;

 * postgresql+psycopg2://@/postgres


,cod_c,cod_m,cod_p,data,hora
0,1,1,1,2016-10-12,14:00:00
1,2,1,4,2016-11-04,12:00:00


A consulta do médico Pedro (`cod_m` = 3) com o paciente Carlos (`cod_p` = 4) passou para uma hora e meia depois


In [ ]:
%%sql
SELECT * 
FROM consultas 
WHERE cod_m=3;

 * postgresql+psycopg2://@/postgres


,cod_c,cod_m,cod_p,data,hora
0,6,3,1,2016-10-19,18:00:00
1,7,3,3,2016-10-12,10:00:00
2,8,3,4,2016-10-19,13:00:00


In [ ]:
%%sql
UPDATE consultas SET hora = '1430' 
WHERE cod_m = 3 AND cod_p = 4;

SELECT * FROM consultas WHERE cod_m=3;

 * postgresql+psycopg2://@/postgres


,cod_c,cod_m,cod_p,data,hora
0,6,3,1,2016-10-19,18:00:00
1,7,3,3,2016-10-12,10:00:00
2,8,3,4,2016-10-19,14:30:00


O funcionário Carlos (`cod_f` = 4) deixou a clínica

In [ ]:
%%sql
SELECT * FROM funcionarios;

 * postgresql+psycopg2://@/postgres


,cod_f,nome,idade,cpf,cidade,salario,cargo
0,1,rita,32,12345678920,sao jose,1200,None
1,2,maria,55,12345678921,palhoca,1220,None
2,3,caio,45,12345678922,florianopolis,1100,None
3,4,carlos,44,12345678923,florianopolis,1200,None
4,5,paula,33,12345678924,florianopolis,2500,None


In [ ]:
%%sql
DELETE FROM funcionarios 
WHERE cod_f = 4;

SELECT * FROM funcionarios;

 * postgresql+psycopg2://@/postgres


,cod_f,nome,idade,cpf,cidade,salario,cargo
0,1,rita,32,12345678920,sao jose,1200,None
1,2,maria,55,12345678921,palhoca,1220,None
2,3,caio,45,12345678922,florianopolis,1100,None
3,5,paula,33,12345678924,florianopolis,2500,None


Os pacientes com câncer ou idade inferior a 24 anos deixaram a clínica


In [ ]:
%%sql
SELECT* FROM pacientes;

 * postgresql+psycopg2://@/postgres


,cod_p,nome,idade,cidade,cpf,doenca
0,1,ana,20,florianopolis,12345678910,gripe
1,3,lucia,30,biguacu,12345678912,tendinite
2,4,carlos,28,joinville,12345678913,sarampo
3,2,paulo,24,ilhota,12345678911,fratura


In [ ]:
%%sql
SELECT* FROM consultas;

 * postgresql+psycopg2://@/postgres


,cod_c,cod_m,cod_p,data,hora
0,1,1,1,2016-10-12,14:00:00
1,3,2,1,2016-10-13,09:00:00
2,4,2,2,2016-10-14,14:00:00
3,5,2,4,2016-10-14,17:00:00
4,6,3,1,2016-10-19,18:00:00
5,7,3,3,2016-10-12,10:00:00
6,9,4,4,2016-10-22,19:30:00
7,2,1,4,2016-11-04,12:00:00
8,8,3,4,2016-10-19,14:30:00


In [ ]:
%%sql
-- Primeiro tem que tirar a dependencia nesta tabela para depois deletar na tabela pacientes
DELETE FROM consultas 
WHERE cod_p = 1; 

DELETE FROM pacientes 
WHERE 
  doenca = 'cancer'
  OR
  idade < 24;

SELECT* FROM pacientes;

 * postgresql+psycopg2://@/postgres


,cod_p,nome,idade,cidade,cpf,doenca
0,3,lucia,30,biguacu,12345678912,tendinite
1,4,carlos,28,joinville,12345678913,sarampo
2,2,paulo,24,ilhota,12345678911,fratura


In [ ]:
%%sql
SELECT * FROM consultas;

 * postgresql+psycopg2://@/postgres


,cod_c,cod_m,cod_p,data,hora
0,4,2,2,2016-10-14,14:00:00
1,5,2,4,2016-10-14,17:00:00
2,7,3,3,2016-10-12,10:00:00
3,9,4,4,2016-10-22,19:30:00
4,2,1,4,2016-11-04,12:00:00
5,8,3,4,2016-10-19,14:30:00


<br/>
<br/>

## **9. Busque as seguintes informações filtrando os dados**

Buscar os dados dos ambulatórios que possuem capacidade superior à 30


In [ ]:
%%sql
SELECT capacidade
FROM ambulatorios
WHERE capacidade > 30;

 * postgresql+psycopg2://@/postgres


,capacidade
0,50
1,40
2,55


Buscar os dados dos médicos com menos de 40 anos ou com especialidade traumatologia


In [ ]:
%%sql
SELECT idade, especialidade
FROM medicos
WHERE 
  idade < 40
  OR 
  especialidade = 'traumatologia';

 * postgresql+psycopg2://@/postgres


,idade,especialidade
0,42,traumatologia
1,28,ortopedia
2,33,neurologia


Buscar os códigos dos médicos e os códigos dos pacientes, para todas as consultas marcadas no período da tarde (a partir das 13h) após o dia 15/10/2016

In [ ]:
%%sql
SELECT cod_m, cod_p
FROM consultas
WHERE 
  hora >= '1300'
  AND 
  data > '2016-10-15';

 * postgresql+psycopg2://@/postgres


,cod_m,cod_p
0,4,4
1,3,4


Buscar o nome e a idade dos pacientes que não residem em Florianópolis

In [ ]:
%%sql
SELECT nome, idade
FROM pacientes
WHERE cidade != 'florianopolis';

 * postgresql+psycopg2://@/postgres


,nome,idade
0,lucia,30
1,carlos,28
2,paulo,24


Buscar o nome e a idade (em meses) dos pacientes

In [ ]:
%%sql
SELECT 
  nome,
  idade*12 
FROM pacientes;

 * postgresql+psycopg2://@/postgres


,nome,?column?
0,lucia,360
1,carlos,336
2,paulo,288


Qual o menor e o maior salário dos funcionários da Florianópolis?

In [ ]:
%%sql
SELECT 
  MIN(salario),
  MAX(salario)
FROM funcionarios
WHERE cidade = 'florianopolis';

 * postgresql+psycopg2://@/postgres


,min,max
0,1100,2500


Qual o horário da última consulta marcada para o dia 13/10/2016?

**NOTE**: reveja o item: `Os pacientes com câncer ou idade inferior a 24 anos deixaram a clínica` 

In [ ]:
%%sql
SELECT hora
FROM consultas
WHERE data = '2016-10-13';

 * postgresql+psycopg2://@/postgres


""


Qual a média de idade dos médicos e o total de ambulatórios atendidos por eles?

In [ ]:
%%sql
SELECT 
  AVG(idade),
  COUNT(DISTINCT nroa)
FROM medicos;

 * postgresql+psycopg2://@/postgres


,avg,count
0,38.8000000000000000,5


Buscar o código, o nome e o salário líquido dos funcionários. O salário líquido é obtido pela diferença entre o salário cadastrado menos 20% deste mesmo salário

In [ ]:
%%sql
SELECT 
  cod_f,
  nome,
  (salario - salario*0.2) AS salario_liq
FROM funcionarios;

 * postgresql+psycopg2://@/postgres


,cod_f,nome,salario_liq
0,1,rita,960.0
1,2,maria,976.0
2,3,caio,880.0
3,5,paula,2000.0


<br/>
<br/>

---

# **JOIN**

## **10. Busque as seguintes informações entre as tabelas**

Buscar o nome dos funcionários que terminam com a letra “a”


In [ ]:
%%sql
SELECT nome
FROM funcionarios
WHERE nome LIKE '%a';

 * postgresql+psycopg2://@/postgres


,nome
0,rita
1,maria
2,paula


Buscar o nome e a especialidade dos médicos cuja segunda e a última letra de seus nomes seja a letra “o”

In [ ]:
%%sql
SELECT 
  nome,
  especialidade
FROM medicos
WHERE nome LIKE '_o%o';

 * postgresql+psycopg2://@/postgres


,nome,especialidade
0,joao,ortopedia


Buscar os códigos e nomes dos pacientes com mais de 25 anos que estão com tendinite, fratura,gripe ou sarampo

In [ ]:
%%sql
SELECT 
  cod_p, 
  nome,
  idade,
  doenca
FROM pacientes
WHERE 
  idade > 25
  AND 
  (
    doenca = 'tendinite'
    OR 
    doenca = 'fratura'
    OR
    doenca = 'gripe'
    OR 
    doenca = 'sarampo'
  );

 * postgresql+psycopg2://@/postgres


,cod_p,nome,idade,doenca
0,3,lucia,30,tendinite
1,4,carlos,28,sarampo


Buscar os `cpf`, `nome` e `idade` de todas as pessoas (`medicos`, `pacientes` ou `funcionarios`) que residem em Florianópolis.

**BIZU:** primeiro junte tudo depois faça o filtro

In [ ]:
%%sql
SELECT 
  cpf,
  nome,
  idade
FROM 
      (
        SELECT cpf, nome, idade
	      FROM pacientes
        UNION
	      SELECT cpf, nome, idade
	      FROM funcionarios
        UNION
	      SELECT cpf, nome, idade
	      FROM medicos
        ) as todas_pessoas;

 * postgresql+psycopg2://@/postgres


,cpf,nome,idade
0,10000100004,marcia,33
1,12345678924,paula,33
2,12345678911,paulo,24
3,12345678912,lucia,30
4,12345678921,maria,55
5,10000100000,joao,40
6,10000100003,carlos,28
7,10000100002,pedro,51
8,12345678913,carlos,28
9,12345678920,rita,32


In [ ]:
%%sql
SELECT 
  cpf,
  nome,
  idade,
  cidade
FROM 
      (
        SELECT cpf, nome, idade, cidade
	      FROM pacientes
        UNION
	      SELECT cpf, nome, idade, cidade
	      FROM funcionarios
        UNION
	      SELECT cpf, nome, idade, cidade
	      FROM medicos
        ) as todas_pessoas
WHERE cidade = 'florianopolis';

 * postgresql+psycopg2://@/postgres


,cpf,nome,idade,cidade
0,12345678924,paula,33,florianopolis
1,10000100000,joao,40,florianopolis
2,12345678922,caio,45,florianopolis


Buscar o número e o andar dos ambulatórios utilizados por médicos ortopedista

In [ ]:
%%sql
SELECT 
  ambulatorios.nroa,
  ambulatorios.andar
FROM 
  ambulatorios JOIN medicos ON ambulatorios.nroa = medicos.nroa
WHERE medicos.nroa=1;

 * postgresql+psycopg2://@/postgres


,nroa,andar
0,1,1


Retornar pares (código, nome) de funcionários e de médicos que residem na mesma cidade (tabela resultado deve ter 4 atributos)

In [ ]:
%%sql
SELECT 
  funcionarios.cod_f,
  funcionarios.nome,
  medicos.cod_m,
  medicos.nome
FROM 
  funcionarios JOIN medicos ON funcionarios.cidade = medicos.cidade;

 * postgresql+psycopg2://@/postgres


,cod_f,nome,cod_m,nome
0,1,rita,3,pedro
1,3,caio,1,joao
2,5,paula,1,joao


Buscar o código e nome dos médicos que possuem consultas marcadas para antes das 12 horas e possuem idade inferior à idade do médico Pedro

In [ ]:
%%sql
-- parte 01: código e nome dos médicos que possuem consultas marcadas para antes das 12 horas
SELECT 
  medicos.cod_m,
  medicos.nome
FROM
  medicos JOIN consultas ON medicos.cod_m = consultas.cod_m
WHERE 
  hora < '1200';

 * postgresql+psycopg2://@/postgres


,cod_m,nome
0,3,pedro


In [ ]:
%%sql
-- parte 02: idade do médico Pedro
SELECT idade 
FROM medicos 
WHERE nome = 'pedro';

 * postgresql+psycopg2://@/postgres


,idade
0,51


In [ ]:
%%sql
-- parte 03: tudo
SELECT 
  medicos.cod_m,
  medicos.nome
FROM 
  medicos JOIN consultas ON medicos.cod_m = consultas.cod_m
WHERE 
  hora < '1200'
  AND 
  idade < (
            SELECT idade 
            FROM medicos 
            WHERE nome = 'pedro'
          );

 * postgresql+psycopg2://@/postgres


""


Buscar o nome e o salário dos funcionários que moram na mesma cidade do funcionário Caio e possuem salário superior ao dele

In [ ]:
%%sql
-- parte 01: nome e o salário dos funcionários que moram na mesma cidade do funcionário Caio
SELECT 
  nome,
  salario
FROM funcionarios
WHERE cidade = (
                  SELECT cidade 
                  FROM funcionarios 
                  WHERE nome='caio'
                );

 * postgresql+psycopg2://@/postgres


,nome,salario
0,caio,1100
1,paula,2500


In [ ]:
%%sql
-- parte 02: tudo
SELECT 
  nome,
  salario
FROM funcionarios
WHERE 
  cidade = (
              SELECT cidade 
              FROM funcionarios 
              WHERE nome='caio'
            )
  AND
  salario > (
              SELECT salario
              FROM funcionarios 
              WHERE nome = 'caio'
            ); 

 * postgresql+psycopg2://@/postgres


,nome,salario
0,paula,2500


Buscar o código, nome e data dos pacientes com consulta marcada para horários após às 14 horas

In [ ]:
%%sql
SELECT 
  pacientes.cod_p,
  pacientes.nome,
  consultas.data
FROM 
  pacientes JOIN consultas ON pacientes.cod_p=consultas.cod_p
WHERE hora > '1400';

 * postgresql+psycopg2://@/postgres


,cod_p,nome,data
0,4,carlos,2016-10-14
1,4,carlos,2016-10-22
2,4,carlos,2016-10-19


Buscar o número e o andar dos ambulatórios cujos médicos possuem consultas marcadas para o dia 12/10/2016

In [ ]:
%%sql
SELECT 
  ambulatorios.nroa,
  ambulatorios.andar
FROM 
  ambulatorios NATURAL JOIN consultas
WHERE data='2016-10-12';

 * postgresql+psycopg2://@/postgres


,nroa,andar
0,1,1
1,2,1
2,3,2
3,4,2
4,5,2


Buscar o nome, CPF e especialidade dos médicos que possuem consultas marcadas com pacientes que estão com tendinite

In [ ]:
%%sql
SELECT 
  medicos.nome,
  medicos.cpf,
  medicos.especialidade
FROM 
  medicos 
    JOIN consultas ON medicos.cod_m=consultas.cod_m
	  JOIN pacientes ON consultas.cod_p=pacientes.cod_p
WHERE doenca = 'tendinite';

 * postgresql+psycopg2://@/postgres


,nome,cpf,especialidade
0,pedro,10000100002,pediatria


Buscar os dados de todos os ambulatórios e, para aqueles ambulatórios onde médicos dão atendimento, exibir também os seus códigos e nomes

In [ ]:
%%sql
SELECT 
  ambulatorios.nroa,
  ambulatorios.andar,
  ambulatorios.capacidade,
  medicos.cod_m AS codigo_medico,
  medicos.nome AS nome_medico
FROM 
  ambulatorios JOIN medicos ON ambulatorios.nroa=medicos.nroa;

 * postgresql+psycopg2://@/postgres


,nroa,andar,capacidade,codigo_medico,nome_medico
0,1,1,30,1,joao
1,2,1,50,2,maria
2,3,2,40,3,pedro
3,4,2,25,4,carlos
4,5,2,55,5,marcia


Buscar o CPF e o nome de todos os médicos e, para aqueles médicos com consultas marcadas, exibir os CPFs e nomes dos seus pacientes e as datas das consultas

In [ ]:
%%sql
SELECT 
  medicos.cpf,
  medicos.nome,
  pacientes.cpf,
  pacientes.nome,
  consultas.data
FROM 
  medicos 
    JOIN consultas ON medicos.cod_m=consultas.cod_m
	  FULL JOIN pacientes ON consultas.cod_p=pacientes.cod_p;

 * postgresql+psycopg2://@/postgres


,cpf,nome,cpf,nome,data
0,10000100001,maria,12345678911,paulo,2016-10-14
1,10000100001,maria,12345678913,carlos,2016-10-14
2,10000100002,pedro,12345678912,lucia,2016-10-12
3,10000100003,carlos,12345678913,carlos,2016-10-22
4,10000100000,joao,12345678913,carlos,2016-11-04
5,10000100002,pedro,12345678913,carlos,2016-10-19


<br/>
<br/>

---

# **SELECT AVANÇADO**

## **11. Busque as seguintes informações entre as tabelas, usando ANY, ALL, GROUP BY, ORDER BY, HAVING e EXISTS**

Retorne Nome e CPF dos médicos que também são pacientes do hospital

In [ ]:
%%sql
-- parte 01: select o que médicos e pacientes tem em comum 
SELECT * 
FROM pacientes;

 * postgresql+psycopg2://@/postgres


,cod_p,nome,idade,cidade,cpf,doenca
0,3,lucia,30,biguacu,12345678912,tendinite
1,4,carlos,28,joinville,12345678913,sarampo
2,2,paulo,24,ilhota,12345678911,fratura


In [ ]:
%%sql
-- parte 01: select o que médicos e pacientes tem em comum 
SELECT * 
FROM medicos;

 * postgresql+psycopg2://@/postgres


,cod_m,nome,idade,especialidade,cpf,cidade,nroa
0,1,joao,40,ortopedia,10000100000,florianopolis,1
1,2,maria,42,traumatologia,10000100001,blumenau,2
2,3,pedro,51,pediatria,10000100002,sao jose,3
3,4,carlos,28,ortopedia,10000100003,joinville,4
4,5,marcia,33,neurologia,10000100004,biguacu,5


In [ ]:
%%sql
-- parte 02: tudo
SELECT 
  medicos.nome,
  medicos.cpf
FROM medicos
WHERE cpf IN (
                SELECT cpf 
                FROM pacientes
              );

 * postgresql+psycopg2://@/postgres


""


Retorne número e andar dos ambulatórios onde nenhum médico dá atendimento

In [ ]:
%%sql
SELECT 
  ambulatorios.nroa,
  ambulatorios.andar
FROM ambulatorios
WHERE ambulatorios.nroa NOT IN (
                                  SELECT medicos.nroa
                                  FROM medicos
                                  WHERE nroa > 0
                                );

 * postgresql+psycopg2://@/postgres


""


Retorne números e andares de todos os ambulatórios, exceto o de menor capacidade

In [ ]:
%%sql
SELECT 
  ambulatorios.nroa,
  ambulatorios.andar
FROM ambulatorios
WHERE 
  ambulatorios.capacidade != ANY(
                                  SELECT MIN(ambulatorios.capacidade) 
                                  FROM ambulatorios
                                 );

 * postgresql+psycopg2://@/postgres


,nroa,andar
0,1,1
1,2,1
2,3,2
3,5,2


Retorne nome e cpf dos pacientes com consultas marcadas para horários anteriores a todos os horários de consultas marcadas para o dia 14/10/2016

In [ ]:
%%sql
SELECT DISTINCT
  pacientes.nome, 
  pacientes.cpf
FROM pacientes
WHERE pacientes.cod_p = ANY(
                              SELECT consultas.cod_p 
                              FROM consultas 
                              WHERE data='2016-10-14'
                            );

 * postgresql+psycopg2://@/postgres


,nome,cpf
0,carlos,12345678913
1,paulo,12345678911


Buscar o nome e o CPF dos médicos que também são pacientes do hospital usando `EXISTS`

In [ ]:
%%sql
SELECT
  medicos.nome,
  medicos.cpf
FROM medicos
WHERE EXISTS(
              SELECT * 
              FROM pacientes 
              WHERE pacientes.cpf = medicos.cpf
            );

 * postgresql+psycopg2://@/postgres


""


Buscar o nome e o CPF dos médicos que têm consultas marcadas com qualquer um paciente. Use `EXISTS`

In [ ]:
%%sql
-- parte 01: Buscar o nome e o CPF dos médicos que têm consultas marcadas
SELECT
  medicos.nome,
  medicos.cpf
FROM 
  medicos JOIN consultas ON medicos.cod_m=consultas.cod_m;

 * postgresql+psycopg2://@/postgres


,nome,cpf
0,maria,10000100001
1,maria,10000100001
2,pedro,10000100002
3,carlos,10000100003
4,joao,10000100000
5,pedro,10000100002


In [ ]:
%%sql
-- parte 02: todos os pacientes com consultas marcadas
SELECT DISTINCT
  pacientes.cod_p
FROM
  pacientes JOIN consultas ON pacientes.cod_p=consultas.cod_p

 * postgresql+psycopg2://@/postgres


,cod_p
0,3
1,4
2,2


In [ ]:
%%sql
-- parte 03: tudo
SELECT DISTINCT 
  medicos.nome,
  medicos.cpf
FROM 
  medicos JOIN consultas ON medicos.cod_m=consultas.cod_m
WHERE EXISTS(
              SELECT DISTINCT
                pacientes.cod_p
              FROM 
                pacientes JOIN consultas ON pacientes.cod_p=consultas.cod_p
            );

 * postgresql+psycopg2://@/postgres


,nome,cpf
0,pedro,10000100002
1,joao,10000100000
2,maria,10000100001
3,carlos,10000100003


Buscar o nome e a cidade dos pacientes que têm consultas marcadas com ortopedistas

In [ ]:
%%sql
-- parte 01: buscar o nome e a cidade dos pacientes que têm consultas marcadas
SELECT DISTINCT
  pacientes.nome,
  pacientes.cidade
FROM 
  pacientes JOIN consultas ON pacientes.cod_p=consultas.cod_p;

 * postgresql+psycopg2://@/postgres


,nome,cidade
0,lucia,biguacu
1,carlos,joinville
2,paulo,ilhota


In [ ]:
%%sql
-- parte 02: medico ortopedista
SELECT medicos.cod_m 
FROM medicos 
WHERE especialidade = 'ortopedia';

 * postgresql+psycopg2://@/postgres


,cod_m
0,1
1,4


In [ ]:
%%sql
-- parte 03: tudo
SELECT DISTINCT
  pacientes.nome,
  pacientes.cidade
FROM 
  pacientes JOIN consultas ON pacientes.cod_p=consultas.cod_p
WHERE 
  consultas.cod_m = (
                      SELECT medicos.cod_m 
                      FROM medicos 
                      WHERE especialidade = 'ortopedia'
                    );

 * postgresql+psycopg2://@/postgres
(psycopg2.ProgrammingError) more than one row returned by a subquery used as an expression

[SQL: -- parte 03: tudo
SELECT DISTINCT
  pacientes.nome,
  pacientes.cidade
FROM 
  pacientes JOIN consultas ON pacientes.cod_p=consultas.cod_p
WHERE 
  consultas.cod_m = (
                      SELECT medicos.cod_m 
                      FROM medicos 
                      WHERE especialidade = 'ortopedia'
                    );]
(Background on this error at: https://sqlalche.me/e/14/f405)


Buscar o nome e o CPF dos médicos que atendem no mesmo ambulatório do médico Pedro

In [ ]:
%%sql
SELECT 
  medicos.nome,
  medicos.cpf
FROM 
  medicos JOIN ambulatorios ON medicos.nroa=ambulatorios.nroa
WHERE 
  medicos.nroa = (
                    SELECT medicos.nroa 
                    FROM medicos 
                    WHERE medicos.nome = 'pedro'
                  );

 * postgresql+psycopg2://@/postgres


,nome,cpf
0,pedro,10000100002


Retorne os dados de todos os funcionários ordenados pelo salário (decrescente) e pela idade (crescente). 

In [ ]:
%%sql
SELECT *
FROM funcionarios
ORDER BY salario DESC, idade ASC;

 * postgresql+psycopg2://@/postgres


,cod_f,nome,idade,cpf,cidade,salario,cargo
0,5,paula,33,12345678924,florianopolis,2500,None
1,2,maria,55,12345678921,palhoca,1220,None
2,1,rita,32,12345678920,sao jose,1200,None
3,3,caio,45,12345678922,florianopolis,1100,None


O nome dos médicos, o número e andar do ambulatório onde eles atendem, ordenado pelo número do ambulatório


In [ ]:
%%sql
SELECT 
  medicos.nome,
  ambulatorios.andar,
  medicos.nroa
FROM 
  medicos JOIN ambulatorios ON medicos.nroa=ambulatorios.nroa
ORDER BY ambulatorios.nroa;

 * postgresql+psycopg2://@/postgres


,nome,andar,nroa
0,joao,1,1
1,maria,1,2
2,pedro,2,3
3,carlos,2,4
4,marcia,2,5


Retorne as datas e o total de consultas em cada data, para horários após às 12 horas

In [ ]:
%%sql
SELECT 
  data,
  count(data)
FROM consultas
WHERE consultas.hora > '1200'
GROUP BY data;

 * postgresql+psycopg2://@/postgres


,data,count
0,2016-10-22,1
1,2016-10-19,1
2,2016-10-14,2


Retorne andares dos ambulatórios e a capacidade total por andar, ordenado pela capacidade (desc)

In [ ]:
%%sql
SELECT
  andar,
  SUM(capacidade) as capacidade_total
FROM ambulatorios
GROUP BY andar
ORDER BY andar ASC;

 * postgresql+psycopg2://@/postgres


,andar,capacidade_total
0,1,80
1,2,120


Retorne os andares dos ambulatórios cuja média de capacidade no andar seja >= 40

In [ ]:
%%sql
SELECT 
  andar,
  AVG(capacidade)
FROM ambulatorios
WHERE capacidade >= 40
GROUP BY andar;

 * postgresql+psycopg2://@/postgres


,andar,avg
0,1,50.0000000000000000
1,2,47.5000000000000000


Excluir os pacientes que não possuem consultas marcadas

In [ ]:
%%sql
DELETE FROM pacientes
WHERE cod_p NOT IN (
                      SELECT cod_p
                      FROM consultas
                   );

 * postgresql+psycopg2://@/postgres


""




O médico Pedro passa a residir na mesma cidade do paciente Paulo

In [ ]:
%%sql
-- parte 01: O médico Pedro reside na mesma cidade do paciente Paulo
SELECT 
  pacientes.cidade 
FROM pacientes 
WHERE 
  pacientes.nome='paulo';

 * postgresql+psycopg2://@/postgres


,cidade
0,ilhota


In [ ]:
%%sql
-- parte 01: O médico Pedro reside na mesma cidade do paciente Paulo
SELECT 
  medicos.cidade 
FROM medicos 
WHERE 
  medicos.nome='pedro';

 * postgresql+psycopg2://@/postgres


,cidade
0,sao jose


In [ ]:
%%sql
-- parte 02: tudo
UPDATE medicos SET cidade = (
                              SELECT pacientes.cidade 
                              FROM pacientes 
                              WHERE pacientes.nome = 'paulo'
                            )
WHERE medicos.nome = 'pedro'

 * postgresql+psycopg2://@/postgres


""


<br/>
<br/>

---

# **WINDOW FUNCTION**

Crie uma coluna `is_duplicatted` onde informe se a coluna `especialidade` é duplicada.

In [ ]:
%%sql
SELECT 
  cod_m,
  nome,
  idade,
  especialidade,
  cpf,
  cidade,
  nroa,
  COUNT(*) OVER(PARTITION BY especialidade) as flag
FROM medicos;

 * postgresql+psycopg2://@/postgres


,cod_m,nome,idade,especialidade,cpf,cidade,nroa,flag
0,5,marcia,33,neurologia,10000100004,biguacu,5,1
1,1,joao,40,ortopedia,10000100000,florianopolis,1,2
2,4,carlos,28,ortopedia,10000100003,joinville,4,2
3,3,pedro,51,pediatria,10000100002,ilhota,3,1
4,2,maria,42,traumatologia,10000100001,blumenau,2,1


In [ ]:
%%sql
SELECT 
  cod_m,
  nome,
  idade,
  especialidade,
  cpf,
  cidade,
  nroa,
  -- COUNT(*) OVER(PARTITION BY especialidade) as flag,

  CASE WHEN (COUNT(*) OVER(PARTITION BY especialidade)) > 1 THEN 'duplicatted'
       ELSE ''
  END is_duplicatted

FROM medicos;

 * postgresql+psycopg2://@/postgres


,cod_m,nome,idade,especialidade,cpf,cidade,nroa,is_duplicatted
0,5,marcia,33,neurologia,10000100004,biguacu,5,
1,1,joao,40,ortopedia,10000100000,florianopolis,1,duplicatted
2,4,carlos,28,ortopedia,10000100003,joinville,4,duplicatted
3,3,pedro,51,pediatria,10000100002,ilhota,3,
4,2,maria,42,traumatologia,10000100001,blumenau,2,


Crie uma coluna `rank_year` onde informe o rank ads idades.

In [ ]:
%%sql
-- não precisa particionar
SELECT 
  cod_m,
  nome,
  idade,
  especialidade,
  cpf,
  cidade,
  nroa,
  RANK() OVER(ORDER BY idade) as rank_idade
FROM medicos;

 * postgresql+psycopg2://@/postgres


,cod_m,nome,idade,especialidade,cpf,cidade,nroa,rank_idade
0,4,carlos,28,ortopedia,10000100003,joinville,4,1
1,5,marcia,33,neurologia,10000100004,biguacu,5,2
2,1,joao,40,ortopedia,10000100000,florianopolis,1,3
3,2,maria,42,traumatologia,10000100001,blumenau,2,4
4,3,pedro,51,pediatria,10000100002,ilhota,3,5


Crie uma coluna `rank_salario` onde informe o rank dos salarios dos `funcionarios`.

In [ ]:
%%sql
-- não precisa particionar
SELECT
  cod_f,
  nome,
  idade,
  cpf,
  cidade,
  salario,
  cargo,
  RANK() OVER(ORDER BY salario) AS rank_salario
FROM funcionarios;

 * postgresql+psycopg2://@/postgres


,cod_f,nome,idade,cpf,cidade,salario,cargo,rank_salario
0,3,caio,45,12345678922,florianopolis,1100,None,1
1,1,rita,32,12345678920,sao jose,1200,None,2
2,2,maria,55,12345678921,palhoca,1220,None,3
3,5,paula,33,12345678924,florianopolis,2500,None,4


---

In [ ]:
%%sql
CREATE TABLE sales(
  name      VARCHAR(100),
	category  VARCHAR(100),
	price     NUMERIC(5,2)
);


--sales
INSERT INTO sales(name, category, price)
VALUES ('Country Living', 'lifestyle', 1.70);

INSERT INTO sales(name, category, price)
VALUES ('News Magazine', 'news', 3.35);

INSERT INTO sales(name, category, price)
VALUES ('Decibel', 'music', 6.50);

INSERT INTO sales(name, category, price)
VALUES ('Drum Magazine', 'music', 6.50);

INSERT INTO sales(name, category, price)
VALUES ('Sunset', 'lifestyle', 12.00);

INSERT INTO sales(name, category, price)
VALUES ('World', 'lifestyle', 9.50);
INSERT INTO sales(name, category, price)
VALUES ('Keyboard', 'music', 8.45);

 * postgresql+psycopg2://@/postgres


""


Rank `price`

In [ ]:
%%sql

SELECT 
  name,
  category,
  price,
  RANK() OVER(ORDER BY price) AS price_rank
FROM sales;

 * postgresql+psycopg2://@/postgres


,name,category,price,price_rank
0,Country Living,lifestyle,1.70,1
1,News Magazine,news,3.35,2
2,Drum Magazine,music,6.50,3
3,Decibel,music,6.50,3
4,Keyboard,music,8.45,5
5,World,lifestyle,9.50,6
6,Sunset,lifestyle,12.00,7


Rank `price` by `category`.

In [ ]:
%%sql
SELECT
    category,
    name,
    price,
    DENSE_RANK() OVER(PARTITION BY category
                      ORDER BY price) AS price_rank
FROM sales;

 * postgresql+psycopg2://@/postgres


,category,name,price,price_rank
0,lifestyle,Country Living,1.70,1
1,lifestyle,World,9.50,2
2,lifestyle,Sunset,12.00,3
3,music,Drum Magazine,6.50,1
4,music,Decibel,6.50,1
5,music,Keyboard,8.45,2
6,news,News Magazine,3.35,1
